In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [111]:
class CustomLinearRegression:
    def __init__(self, fit_intercept=True, normalize=False):
        self.fit_intercept = fit_intercept
        self.normalize = normalize
    
    def _check(self, X, y):
        if not isinstance(X, list):
            raise ValueError('X is not a list')
        elif not isinstance(y, list):
            raise ValueError('y is not a list')
        else:
            X_array = np.array(X)
            y_array = np.array(y)
        
        if X_array.shape[0] != y_array.shape[0]:
            raise ValueError(f'number of rows in X and y is not equal')

        self.X = X_array
        self.y = y_array
        
    def _add_intercept(self):     
        if self.fit_intercept:
            x0_array = np.ones([self.X.shape[0], 1])
            self.X = np.concatenate((x0_array, self.X), axis=1)
            
    def _make_normalized(self):
        if self.normalize:
            norm = np.linalg.norm(self.X, axis=1, keepdims=True)
            print(norm)
            self.X = self.X / norm
            
    def _compute_loss(self):
        m = self.X.shape[0]
        h = np.dot(self.X, self.theta)
        loss =np.sum((h - y) ** 2) / (2 * m)
        
        return loss
    
    def _compute_hypothesis(self):
        return np.dot(self.X, self.theta)
            
    def _descend_gradient(self, alpha, num_iter):
        m = self.X.shape[0]
        num_features = self.X.shape[1]
        self.theta = np.zeros((num_features, 1))
        
        for i in range(num_iter):
            h = self._compute_hypothesis()
            error = h - y
            temp_X = np.dot(error.T, self.X)
            self.theta = self.theta - (alpha / m)  * (temp_X.T)

    def fit(self, X, y):
        # checking X and y and making numpy arrays
        self._check(X, y)

        # if normalize add normalization
        self._make_normalized()
        
        # if fit_intercept adding column with one numbers
        self._add_intercept()
        
        self._descend_gradient(alpha=0.01, num_iter=1000)
        
        return self.theta
    
    def predict(self, x_test):
        x_test = np.array(x_test)
        x0_array = np.ones([self.X.shape[0], 1])
        x_test = np.concatenate((x0_array, x_test), axis=1)
        print(x_test)
        return np.dot(x_test, self.theta)
    
    def score(self, X, y):
        

In [114]:
x = [[1,2], [2,5], [5, 10]]
y = [[2], [4], [7]]

model = CustomLinearRegression(normalize=True)
model.fit(x, y)

[[ 2.23606798]
 [ 5.38516481]
 [11.18033989]]


array([[2.16773024],
       [0.99785882],
       [1.92611424]])

In [115]:
model.predict(x)

[[ 1.  1.  2.]
 [ 1.  2.  5.]
 [ 1.  5. 10.]]


array([[ 7.01781753],
       [13.79401906],
       [26.4181667 ]])